In [6]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm
import xgboost
import sklearn
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight

In [7]:
df_X = pd.read_csv("data/X_train_features_final.csv", index_col=0)
df_Y = pd.read_csv("data/y_train.csv", index_col="id")

In [8]:
from sklearn.model_selection import RandomizedSearchCV

random_grid = {'C': [0.1,1, 10, 100], 
                'gamma': [1,0.1,0.01,0.001],
                'kernel': ['rbf', 'poly', 'sigmoid']
    }
print(random_grid)

{'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'poly', 'sigmoid']}


In [9]:
svc = SVC(class_weight = 'balanced')
svc_random = RandomizedSearchCV(estimator = svc, param_distributions = random_grid, scoring='f1_micro', n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)

In [10]:
X = df_X.to_numpy()
Y = df_Y.to_numpy().squeeze()
svc_random.fit(X, Y)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   4.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.8s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   6.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   6.4s
[CV] END ...................

RandomizedSearchCV(cv=10, estimator=SVC(class_weight='balanced'), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100],
                                        'gamma': [1, 0.1, 0.01, 0.001],
                                        'kernel': ['rbf', 'poly', 'sigmoid']},
                   random_state=42, scoring='f1_micro', verbose=2)

In [11]:
svc_random.best_params_

{'kernel': 'rbf', 'gamma': 0.001, 'C': 10}

In [14]:
# best params
best_params = {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}

Evaluate

In [12]:
skf = StratifiedKFold(n_splits=5)
f1_base = []
f1_optim = []

for train_index, test_index in skf.split(X, Y):
    clf = SVC(class_weight = 'balanced')
    
    clf.fit(X[train_index], Y[train_index])
    y_pred = clf.predict(X[test_index])
    f1_base.append(f1_score(Y[test_index], y_pred, average='micro'))
    
    clf_optim = svc_random.best_estimator_
    clf_optim.fit(X[train_index], Y[train_index])
    y_pred = clf_optim.predict(X[test_index])
    f1_optim.append(f1_score(Y[test_index], y_pred, average='micro'))

In [13]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.7392988605816229
0.8008612460288369
